# Qwen3 Embeddingek EDA - CourtRankRL projekt

Ez a notebook a qwen_embedding_runpod.ipynb által generált FAISS indexben tárolt embeddingeket elemzi.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import json
import faiss
from pathlib import Path

# Plot stílus beállítása
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
try:
    from configs import config
except ImportError:
    print("Figyelem: configs modul nem található. Használja a default értékeket.")
    class Config:
        FAISS_INDEX_PATH = project_root / "data" / "index" / "faiss_index.bin"
        CHUNK_ID_MAP_PATH = project_root / "data" / "index" / "chunk_id_map.json"
    config = Config()

print("FAISS index és chunk adatok betöltése...")

## 1. FAISS Index betöltése és embeddingek kivonása

In [ ]:
# FAISS index betöltése
faiss_path = config.FAISS_INDEX_PATH
chunk_map_path = config.CHUNK_ID_MAP_PATH

if faiss_path and Path(faiss_path).exists():
    try:
        index = faiss.read_index(str(faiss_path))
        print(f"✅ FAISS index betöltve: {index.ntotal} vektor, {index.d} dimenzió")

        # Embeddingek kivonása (minden vektor)
        embeddings = []
        for i in range(index.ntotal):
            embedding = index.reconstruct(i)
            embeddings.append(embedding)

        embeddings = np.array(embeddings)
        print(f"✅ Embeddingek kivonva: {embeddings.shape}")

    except Exception as e:
        print(f"❌ Hiba a FAISS index betöltése során: {e}")
        index = None
        embeddings = None
else:
    print(f"⚠️ FAISS index nem található: {faiss_path}")
    print("Futtassa a qwen_embedding_runpod.ipynb-t először!")

# Chunk ID mapping betöltése
if chunk_map_path and Path(chunk_map_path).exists():
    try:
        with open(chunk_map_path, 'r', encoding='utf-8') as f:
            chunk_id_map = json.load(f)
        print(f"✅ Chunk ID map betöltve: {len(chunk_id_map)} mapping")
    except Exception as e:
        print(f"❌ Hiba a chunk ID map betöltése során: {e}")
        chunk_id_map = None
else:
    print(f"⚠️ Chunk ID map nem található: {chunk_map_path}")

## 2. Chunk adatok betöltése

In [ ]:
# Chunkok betöltése mintavételezéssel
df = None
chunks_file = getattr(config, 'CHUNKS_JSONL', None)

if chunks_file and Path(chunks_file).exists() and index is not None and chunk_id_map is not None:
    try:
        sample_size = min(5000, index.ntotal)  # Maximum 5000 chunk elemzésre
        print(f"📊 Mintavétel: {sample_size} chunk betöltése...")

        chunks_list = []
        chunk_ids = list(chunk_id_map.values())[:sample_size]

        with open(chunks_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    chunk = json.loads(line.strip())
                    if chunk.get('chunk_id') in chunk_ids:
                        chunks_list.append(chunk)
                except json.JSONDecodeError:
                    continue
                if len(chunks_list) >= sample_size:
                    break

        if chunks_list:
            df = pd.DataFrame(chunks_list)
            print(f"✅ Betöltött chunkok száma: {len(df)}")

            # Embeddingek hozzárendelése a chunkokhoz
            if embeddings is not None:
                try:
                    embedding_dict = {chunk_id_map[str(i)]: embeddings[i] for i in range(len(embeddings))}
                    df = df.assign(embedding=df['chunk_id'].map(embedding_dict))
                    valid_embeddings = df['embedding'].notna().sum()
                    print(f"✅ Embeddingek hozzárendelve: {valid_embeddings} chunk")
                except Exception as e:
                    print(f"❌ Hiba az embeddingek hozzárendelése során: {e}")
            else:
                print("⚠️ Nincsenek embeddingek a hozzárendeléshez")
        else:
            print("⚠️ Nem találhatóak megfelelő chunkok")

    except Exception as e:
        print(f"❌ Hiba a chunkok betöltése során: {e}")
        df = None
else:
    print("⚠️ Hiányzó adatok a chunk betöltéshez:")
    if not chunks_file:
        print("  - CHUNKS_JSONL konfiguráció")
    if not Path(chunks_file).exists() if chunks_file else None:
        print(f"  - Fájl nem található: {chunks_file}")
    if index is None:
        print("  - FAISS index")
    if chunk_id_map is None:
        print("  - Chunk ID map")

## 3. Embeddingek alapellenőrzése

In [ ]:
if not df.empty and 'embedding' in df.columns:
    print("Embeddingek alapvető statisztikái:")
    
    # Első embedding típusa és dimenziója
    first_embedding = df['embedding'].iloc[0]
    print(f"Első embedding típusa: {type(first_embedding)}")
    print(f"Első embedding dimenziója: {len(first_embedding)}")
    
    # Embeddingek normái (L2 normalizálás ellenőrzése)
    norms = df['embedding'].apply(lambda x: np.linalg.norm(x))
    print(f"Embedding normák statisztikái:")
    print(norms.describe())
    
    # Normalizálás ellenőrzése
    normalized_count = norms.apply(lambda x: abs(x - 1.0) < 0.01).sum()
    print(f"L2-normalizált embeddingek: {normalized_count}/{len(df)} ({100*normalized_count/len(df):.1f}%)")
    
    # Hiányzó embeddingek
    missing_embeddings = df['embedding'].isna().sum()
    print(f"Hiányzó embeddingek: {missing_embeddings}")
    
else:
    print("Nincs embedding adat az elemzéshez.")

## 4. PCA dimenziócsökkentés és vizualizáció

In [ ]:
if not df.empty and 'embedding' in df.columns:
    # Hiányzó embeddingek eltávolítása
    valid_df = df.dropna(subset=['embedding']).copy()
    
    if len(valid_df) > 100:  # Minimum 100 embedding PCA-hoz
        # Embeddingek NumPy tömbbé alakítása
        X = np.vstack(valid_df['embedding'].values)
        print(f"PCA bemenet: {X.shape}")
        
        # PCA futtatása
        pca = PCA(n_components=2)
        X_reduced = pca.fit_transform(X)
        
        print(f"PCA első két komponens varianciája: {pca.explained_variance_ratio_}")
        print(f"Összes magyarázott variancia: {sum(pca.explained_variance_ratio_):.3f}")
        
        # PCA eredmény vizualizáció
        plt.figure(figsize=(10, 7))
        plt.scatter(X_reduced[:, 0], X_reduced[:, 1], s=2, alpha=0.6)
        plt.title("Qwen3 Embeddingek PCA 2D leképezése")
        plt.xlabel("Főkomponens 1")
        plt.ylabel("Főkomponens 2")
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Jogterület szerinti színezés
        if 'domain' in valid_df.columns:
            plt.figure(figsize=(12, 8))
            domains = valid_df['domain'].fillna('ismeretlen').values
            scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=valid_df['domain'].astype('category').cat.codes, s=8, alpha=0.6, cmap='tab10')
            plt.title("Qwen3 Embeddingek jogterület szerint színezve")
            plt.xlabel("Főkomponens 1")
            plt.ylabel("Főkomponens 2")
            plt.legend(handles=scatter.legend_elements()[0], labels=valid_df['domain'].astype('category').cat.categories.tolist(), bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.grid(True, alpha=0.3)
            plt.show()
        
        # Bíróság szerinti színezés
        if 'court' in valid_df.columns:
            plt.figure(figsize=(12, 8))
            courts = valid_df['court'].fillna('ismeretlen').values
            scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=valid_df['court'].astype('category').cat.codes, s=8, alpha=0.6, cmap='Set3')
            plt.title("Qwen3 Embeddingek bíróság szerint színezve")
            plt.xlabel("Főkomponens 1")
            plt.ylabel("Főkomponens 2")
            plt.legend(handles=scatter.legend_elements()[0], labels=valid_df['court'].astype('category').cat.categories.tolist(), bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.grid(True, alpha=0.3)
            plt.show()
    else:
        print(f"Túl kevés embedding PCA-hoz: {len(valid_df)}")
else:
    print("Nincs embedding adat a PCA-hoz.")

## 5. Embedding minőségi metrikák

In [ ]:
if not df.empty and 'embedding' in df.columns:
    valid_df = df.dropna(subset=['embedding']).copy()
    
    if len(valid_df) > 10:
        X = np.vstack(valid_df['embedding'].values)
        
        print("Embedding minőségi metrikák:")
        
        # Átlagos norma (már ellenőriztük)
        norms = np.linalg.norm(X, axis=1)
        print(f"Normák átlaga: {norms.mean():.4f}")
        print(f"Normák szórása: {norms.std():.4f}")
        
        # Embeddingek közötti távolságok
        # Véletlenszerűen kiválasztott 1000 pár távolsága
        n_pairs = min(1000, len(X) * (len(X) - 1) // 2)
        indices = np.random.choice(len(X), size=min(len(X), 100), replace=False)
        
        distances = []
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                dist = np.linalg.norm(X[indices[i]] - X[indices[j]])
                distances.append(dist)
        
        distances = np.array(distances)
        print(f"Embeddingek közötti átlagos távolság: {distances.mean():.4f}")
        print(f"Embeddingek közötti távolság szórása: {distances.std():.4f}")
        
        # Legközelebbi és legtávolabbi pontok
        print(f"Legkisebb távolság: {distances.min():.4f}")
        print(f"Legnagyobb távolság: {distances.max():.4f}")
        
        # Embeddingek sűrűsége
        print(f"\nEmbedding dimenzió: {X.shape[1]}")
        print(f"Adatpontok száma: {len(X)}")
        print(f"Adatpontok sűrűsége: {len(X) / X.shape[1]:.2f}")
        
    else:
        print(f"Túl kevés embedding a minőségi metrikákhoz: {len(valid_df)}")
else:
    print("Nincs embedding adat a minőségi metrikákhoz.")

## 6. Következtetések